In [2]:
from model.yolov4 import TinyYoloV4

In [3]:
model = TinyYoloV4(num_classes=20)

In [6]:
from torchsummary import summary
summary(model, input)

Layer (type:depth-idx)                   Output Shape              Param #
├─CSPTiny: 1-1                           [-1, 512, 13, 13]         --
|    └─Conv2dBatchLeaky: 2-1             [-1, 32, 208, 208]        --
|    |    └─Sequential: 3-1              [-1, 32, 208, 208]        928
|    └─Conv2dBatchLeaky: 2-2             [-1, 64, 104, 104]        --
|    |    └─Sequential: 3-2              [-1, 64, 104, 104]        18,560
|    └─CSPBlock: 2-3                     [-1, 128, 104, 104]       --
|    |    └─Conv2dBatchLeaky: 3-3        [-1, 64, 104, 104]        4,224
|    |    └─Conv2dBatchLeaky: 3-4        [-1, 128, 104, 104]       73,984
|    |    └─Conv2dBatchLeaky: 3-5        [-1, 64, 104, 104]        8,320
|    |    └─Conv2dBatchLeaky: 3-6        [-1, 64, 104, 104]        8,320
|    |    └─Conv2dBatchLeaky: 3-7        [-1, 64, 104, 104]        4,224
|    |    └─Conv2dBatchLeaky: 3-8        [-1, 64, 104, 104]        36,992
|    |    └─Conv2dBatchLeaky: 3-9        [-1, 64, 104, 104] 

Layer (type:depth-idx)                   Output Shape              Param #
├─CSPTiny: 1-1                           [-1, 512, 13, 13]         --
|    └─Conv2dBatchLeaky: 2-1             [-1, 32, 208, 208]        --
|    |    └─Sequential: 3-1              [-1, 32, 208, 208]        928
|    └─Conv2dBatchLeaky: 2-2             [-1, 64, 104, 104]        --
|    |    └─Sequential: 3-2              [-1, 64, 104, 104]        18,560
|    └─CSPBlock: 2-3                     [-1, 128, 104, 104]       --
|    |    └─Conv2dBatchLeaky: 3-3        [-1, 64, 104, 104]        4,224
|    |    └─Conv2dBatchLeaky: 3-4        [-1, 128, 104, 104]       73,984
|    |    └─Conv2dBatchLeaky: 3-5        [-1, 64, 104, 104]        8,320
|    |    └─Conv2dBatchLeaky: 3-6        [-1, 64, 104, 104]        8,320
|    |    └─Conv2dBatchLeaky: 3-7        [-1, 64, 104, 104]        4,224
|    |    └─Conv2dBatchLeaky: 3-8        [-1, 64, 104, 104]        36,992
|    |    └─Conv2dBatchLeaky: 3-9        [-1, 64, 104, 104] 

In [5]:
import torch
input = torch.randn((1, 3, 416, 416))

In [5]:
o = model(input)

In [6]:
o[0].shape, o[1].shape

(torch.Size([1, 13, 13, 3, 25]), torch.Size([1, 26, 26, 3, 25]))

In [8]:
#loss test

In [11]:
from thop import profile
macs, params = profile(model, inputs=(input, ))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.


In [12]:
from thop import clever_format
macs, params = clever_format([macs, params], "%.3f")

In [13]:
print(macs, params)

8.923G 16.379M


In [1]:
from model.loss import *

In [2]:
l = YoloV4_Loss()

In [3]:
g = torch.Generator().manual_seed(0)

preds  = [torch.randn((32, s,s,3,25),generator=g) for s in S]
ground_truths =[torch.randn((32, s,s,3,25),generator=g) for s in S]

for ground_truth in ground_truths:
  ground_truth[..., 0] = torch.empty_like(ground_truth[..., 0], dtype=torch.long).random_(2)
  ground_truth[...,5:] = torch.empty_like(ground_truth[..., 5:], dtype=torch.long).random_(2)

loss = YoloV4_Loss()
print(loss(preds, ground_truths))

tensor(6.5671)
tensor(6.5763)
tensor(13.1435)
